<a href="https://colab.research.google.com/github/arindalestari/LAB-AI/blob/main/Kelompok_5_Tugas_LAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Kelompok 5**

Nama Anggota:
1.	Arinda Cinta Lestari (2802559276)
2.	Choirul Wafa (2802545485)
3.	Muhammad Amir Azis (2802553575)
4.	Muhammad Rafi Zidane (2502044932)
5.	Syendi Endina Yohana Kale (2401979675)

**Group Assignment**

Objective: Build and evaluate a BERT classifier for an Indonesian dataset.

Tasks:

1. Find or prepare an Indonesian sentiment dataset (e.g., product reviews, tweets).

2. Tokenize using a multilingual or Indonesian-specific BERT model (indobenchmark/indobert-base-p1).

3. Fine-tune the model, adapting hyperparameters for data size.

4. Evaluate with accuracy, precision/recall, and confusion matrix.

5. Analyze common error cases and discuss language-specific challenges

**Deliverables:**

1. Jupyter notebook containing the full pipeline.

2. Presentation slide deck summarizing your findings and challenges

# **Pendahuluan**

Dataset yang digunakan berasal dari link berikut: https://www.kaggle.com/datasets/dionisiusdh/covid19-indonesian-twitter-sentiment/data.

Dataset ini berisi Tweet dalam bahasa Indonesia dari pengguna yang telah menggunakan kata kunci berikut: "Corona dan Pemerintah" atau "Covid dan Pemerintah" (Pemerintah = Pemerintah) dari Mei hingga Juli 2020. Dataset ini terdiri dari berbagai variabel yang terkait dengan Twitter: tweet itu sendiri, akun yang mengirim tweet, hashtag yang digunakan, waktu, beberapa ID, dan lainnya.

# **Read Data**

In [4]:
import kagglehub
dionisiusdh_covid19_indonesian_twitter_sentiment_path = kagglehub.dataset_download('dionisiusdh/covid19-indonesian-twitter-sentiment')

print('Data source import complete.')

Data source import complete.


In [5]:
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Membaca file CSV dari path yang benar
file_path = '/kaggle/input/covid19-indonesian-twitter-sentiment/covid-sentiment.csv'
data = pd.read_csv(file_path)

# Cek beberapa baris pertama dari dataset
data.head(5)# Melihat informasi tentang dataset

,conversation_id,date,time,user_id,username,tweet,mentions,replies_count,retweets_count,likes_count,hashtags
0,1258425982907637761,2020-05-07,23:58:29,1058474317,monologis_id,https://bit.ly/2L6CcbB | Seharusnya saat ini...,[],0,3,0,[]
1,1258320972198940675,2020-05-07,23:57:30,1179769476,its_dul,Klo kata gw Pemerintah tuh lagi menerapkan Her...,['mas__piyuuu'],0,0,0,[]
2,1258356644427083777,2020-05-07,23:53:20,1012156669831229441,meonkbaong,Saat ini yang bisa saya lakukan hanya menyiapk...,['oiivert'],0,0,0,[]
3,1258424368993931265,2020-05-07,23:52:04,1204303690061844481,rakyatdotnews,"Satu Warga Positif Corona, Bupati: Kondisi ini...",[],0,0,0,"['#rakyatdotnews', '#referensiterkini', '#raky..."
4,1258423545698246656,2020-05-07,23:48:48,227620381,annisathalib_,"emosi bgt, lg kondisi begini gue disuruh liput...",[],0,0,0,['#dirumahaja']


In [6]:
print(data.info()) # Melihat tipe data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52959 entries, 0 to 52958
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   conversation_id  52959 non-null  int64 
 1   date             52959 non-null  object
 2   time             52959 non-null  object
 3   user_id          52959 non-null  int64 
 4   username         52959 non-null  object
 5   tweet            52959 non-null  object
 6   mentions         52959 non-null  object
 7   replies_count    52959 non-null  int64 
 8   retweets_count   52959 non-null  int64 
 9   likes_count      52959 non-null  int64 
 10  hashtags         52959 non-null  object
dtypes: int64(5), object(6)
memory usage: 4.4+ MB
None


# **Preprocessing Data**



In [7]:
# Menghapus URL dan karakter baru baris (newline)
data['tweet'] = data['tweet'].str.replace(r'http\S+', '', regex=True)  # Menghapus URL
data['tweet'] = data['tweet'].str.replace(r'\n', ' ', regex=True)  # Menghapus newline

Langkah-langkah preprocessing ini bertujuan untuk membersihkan teks tweet. Pertama, URL dihapus menggunakan pola r'http\S+' karena URL tidak relevan untuk analisis teks. Kemudian, karakter newline (\n) dihapus dan diganti dengan spasi agar tweet tetap berada dalam satu baris, memudahkan pemrosesan data lebih lanjut.

# **Tokenisasi**

In [8]:
tokenizer = AutoTokenizer.from_pretrained('indobenchmark/indobert-base-p1')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [9]:
# Fungsi tokenisasi
def tokenize_function(examples):
    return tokenizer(examples['tweet'], padding='max_length', truncation=True)

# Memisahkan data menjadi data latih dan data uji
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Melakukan tokenisasi
train_encodings = tokenizer(list(train_data['tweet']), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_data['tweet']), truncation=True, padding=True, max_length=128)

Kode ini melakukan tokenisasi pada teks tweet dengan menggunakan tokenizer untuk mengubahnya menjadi format yang bisa dipahami oleh model. Fungsi tokenize_function digunakan untuk memproses tweet dengan padding dan truncation. Selanjutnya, data dibagi menjadi 80% untuk pelatihan dan 20% untuk pengujian, lalu tokenisasi dilakukan pada kedua set data (train_data dan test_data) dengan panjang maksimal 128 token, agar siap untuk pelatihan model.

# **Trainer Dataset**

In [10]:
# Menambahkan label manual karena belum ada (asumsi positif=2, netral=1, negatif=0)
data['label'] = data['tweet'].apply(lambda x: 2 if 'positif' in x else (1 if 'netral' in x else 0))

# Memisahkan dataset menjadi data latih dan data uji
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Melanjutkan dengan tokenisasi dan pelatihan model
train_encodings = tokenizer(list(train_data['tweet']), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_data['tweet']), truncation=True, padding=True, max_length=128)

train_labels = train_data['label'].values
test_labels = test_data['label'].values

import torch
from torch.utils.data import Dataset

# Mendefinisikan kelas SentimentDataset
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Siapkan dataset untuk Trainer
train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)

# **Fine Tunning Model with Trainer**

In [13]:
pip install --upgrade transformers

In [11]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

# Memuat model BERT untuk bahasa Indonesia
model = AutoModelForSequenceClassification.from_pretrained("indobenchmark/indobert-base-p1", num_labels=3)  # 3 label: Negatif, Netral, Positif

# Memuat tokenizer yang sesuai dengan model
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")

# Tokenisasi data
# Pastikan dataset 'train_data' dan 'test_data' sudah ada dan berisi tweet
train_encodings = tokenizer(list(train_data['tweet']), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_data['tweet']), truncation=True, padding=True, max_length=128)

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

In [19]:
training_args = TrainingArguments(
    output_dir='./results',           # Direktori untuk menyimpan hasil
    num_train_epochs=1,               # Jumlah epoch pelatihan (lebih sedikit)
    per_device_train_batch_size=8,    # Ukuran batch untuk pelatihan
    per_device_eval_batch_size=16,    # Ukuran batch untuk evaluasi
    warmup_steps=25,                  # Jumlah langkah untuk warmup yang lebih sedikit
    weight_decay=0.01,                # Penurunan bobot untuk mencegah overfitting
    logging_dir='./logs',             # Direktori untuk log
    logging_steps=10,                 # Menampilkan log setiap 10 langkah
    eval_steps=25,                   # Evaluasi setiap 25 langkah
    save_steps=25,                   # Menyimpan model setiap 25 langkah
    save_strategy="steps",            # Menyimpan model pada langkah tertentu
)

# **Evaluasi Model**

In [20]:
# Menyiapkan Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Use train_dataset instead of train_encodings
    eval_dataset=test_dataset,    # Use test_dataset instead of test_encodings
    compute_metrics=lambda p: {
        'accuracy': accuracy_score(p.predictions.argmax(axis=-1), p.label_ids),
        'precision': precision_recall_fscore_support(p.predictions.argmax(axis=-1), p.label_ids, average='weighted')[0],
        'recall': precision_recall_fscore_support(p.predictions.argmax(axis=-1), p.label_ids, average='weighted')[1],
        'f1': precision_recall_fscore_support(p.predictions.argmax(axis=-1), p.label_ids, average='weighted')[2]
    }
)

# Melatih model
trainer.train()

# Evaluasi Model
eval_results = trainer.evaluate()
print("Hasil Evaluasi:", eval_results)

Step,Training Loss
10,0.175900
20,0.272800
30,0.021200
40,0.159900
50,0.138500
60,0.159800
70,0.295900
80,0.130100
90,0.008000
100,0.292300


KeyboardInterrupt: 

In [21]:
# Menyimpan model setelah beberapa langkah
trainer.save_model('./model_checkpoint')

In [22]:
# Evaluasi model setelah pelatihan
eval_results = trainer.evaluate()
print("Hasil Evaluasi:", eval_results)

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Hasil Evaluasi: {'eval_loss': 0.18427279591560364, 'eval_accuracy': 0.9655400302114804, 'eval_precision': 1.0, 'eval_recall': 0.9655400302114804, 'eval_f1': 0.9824679379413036}


Hasil Analisa:

Berdasarkan hasil evaluasi model, dapat disimpulkan bahwa model analisis sentimen yang telah dilatih menunjukkan kinerja yang sangat baik. Model ini berhasil mencapai akurasi tinggi sebesar 96.55%, yang menunjukkan bahwa model mampu memprediksi dengan benar sebagian besar contoh pada dataset uji. Selain itu, model memiliki presisi yang sempurna 1.0, yang berarti bahwa setiap prediksi positif yang dibuat oleh model adalah benar (tanpa kesalahan dalam mengklasifikasikan sentimen positif). Meskipun ada sedikit kesalahan dalam mendeteksi beberapa contoh positif, model menunjukkan recall 96.55%, yang



> **Kendala**

Selama proses pelatihan dan pengujian model ini, terdapat beberapa kendala yang cukup signifikan, di antaranya keterbatasan waktu dan ukuran dataset yang sangat besar. Hal ini menyebabkan waktu runtime menjadi sangat lama, sehingga mempengaruhi efisiensi keseluruhan. Untuk mengatasi kendala ini, saya terpaksa menghentikan beberapa langkah evaluasi guna mempercepat proses dan memungkinkan untuk melanjutkan ke tahapan selanjutnya. Meskipun langkah-langkah tersebut dihentikan lebih awal, hasil evaluasi tetap dapat diperoleh dengan mengevaluasi subset data, dan pelatihan model bisa dilanjutkan menggunakan data yang lebih kecil atau dengan menggunakan perangkat keras yang lebih kuat seperti GPU.

# **Confussion Matrix**

In [23]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Mendapatkan prediksi dari dataset uji
predictions = trainer.predict(test_dataset)
y_pred = predictions.predictions.argmax(axis=-1)

# Matriks kebingunguan
cm = confusion_matrix(test_data['label'], y_pred)

# Menampilkan matriks kebingunguan dengan heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negatif', 'Netral', 'Positif'], yticklabels=['Negatif', 'Netral', 'Positif'])
plt.xlabel('Prediksi')
plt.ylabel('Asli')
plt.title('Matriks Konfusi')
plt.show()

KeyboardInterrupt: 

In [24]:
y_pred = predictions.predictions.argmax(axis=-1)
# Matriks kebingunguan
cm = confusion_matrix(test_data['label'], y_pred)

# Menampilkan matriks kebingunguan dengan heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negatif', 'Netral', 'Positif'], yticklabels=['Negatif', 'Netral', 'Positif'])
plt.xlabel('Prediksi')
plt.ylabel('Asli')
plt.title('Matriks Konfusi')
plt.show()

NameError: name 'predictions' is not defined

In [25]:
# Analisis Kesalahan
# Mengambil data yang salah diklasifikasikan
incorrect_predictions = test_data.iloc[y_pred != test_data['label']]

# Menampilkan beberapa contoh kesalahan klasifikasi
print("Beberapa contoh kesalahan klasifikasi:")
for i in range(min(5, len(incorrect_predictions))):  # Menampilkan 5 contoh kesalahan
    print(f"Teks: {incorrect_predictions.iloc[i]['tweet']}")
    print(f"Label Asli: {incorrect_predictions.iloc[i]['label']}")
    print(f"Label Prediksi: {y_pred[incorrect_predictions.index[i]]}")
    print("-" * 50)

NameError: name 'y_pred' is not defined